In [2]:
import pandas as pd

In [24]:
df = pd.read_csv('data/flights.csv').sample(frac=0.1).reset_index(drop=True)
df_airports = pd.read_csv('data/airports.csv')
df_airlines = pd.read_csv('data/airlines.csv')

/var/folders/07/9l9kqd013xjdlz8_0b2__8xr0000gq/T/ipykernel_98598/617258117.py:1: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/flights.csv').sample(frac=0.1).reset_index(drop=True)


In [25]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# some cleaning
df = df[['ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'DEPARTURE_DELAY', 'ARRIVAL_DELAY', 'AIRLINE', 'DISTANCE']]

df = df.dropna()
# discard flights with airport codes that are not in the airports dataset
df = df[df['ORIGIN_AIRPORT'].isin(df_airports['IATA_CODE'])]
# discard flights with airline codes that are not in the airlines dataset
df = df[df['AIRLINE'].isin(df_airlines['IATA_CODE'])]
df = df.sample(frac=0.1, random_state=1)

In [26]:
df['ORIGIN_AIRPORT'] = pd.Categorical(df['ORIGIN_AIRPORT'])
df['DESTINATION_AIRPORT'] = pd.Categorical(df['DESTINATION_AIRPORT'])
df['AIRLINE'] = pd.Categorical(df['AIRLINE'])

df['ORIGIN_AIRPORT'] = df['ORIGIN_AIRPORT'].cat.codes
df['DESTINATION_AIRPORT'] = df['DESTINATION_AIRPORT'].cat.codes
df['AIRLINE'] = df['AIRLINE'].cat.codes

In [27]:
df

,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,ARRIVAL_DELAY,AIRLINE,DISTANCE
220351,149,189,-6.0,-17.0,8,854
68305,236,140,-3.0,-35.0,0,2917
471781,246,312,17.0,11.0,1,31
193398,141,37,-5.0,-7.0,13,670
288491,181,273,140.0,154.0,2,354
...,...,...,...,...,...,...
292947,207,20,0.0,-5.0,4,302
257319,279,165,-9.0,-11.0,1,95
186137,233,234,-6.0,4.0,11,1009
548834,225,200,25.0,8.0,9,67


In [28]:
X = df[['ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'DEPARTURE_DELAY', 'AIRLINE', 'DISTANCE']]
y = df['ARRIVAL_DELAY']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [40]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

# Create a DataLoader
train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_data = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_data, batch_size=64, shuffle=True)

Define the model

In [68]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class FlightDelayModel(nn.Module):
    def __init__(self, node_feature_dim, edge_feature_dim):
        super(FlightDelayModel, self).__init__()
        self.conv1 = GCNConv(node_feature_dim, 128)
        self.conv2 = GCNConv(128, 32)
        self.edge_pred = torch.nn.Linear(edge_feature_dim, 1)
        self.dropout = torch.nn.Dropout(0.5)
        
    def forward(self, x, edge_index, edge_features):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.edge_pred(torch.cat([x[edge_index[0]], x[edge_index[1]], edge_features], dim=1))
        return x

In [69]:
model = FlightDelayModel(X_train_tensor.shape[1], 1)
criterion = nn.MSELoss()  # Mean Squared Error Loss for regression tasks
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 10
for epoch in range(epochs):
    model.train()
    for batch in train_loader:  # Assume you have a DataLoader `train_loader`
        optimizer.zero_grad()   
        numerical_data = batch[0]
        targets = batch[1]
        outputs = model(numerical_data)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')


TypeError: FlightDelayModel.forward() missing 2 required positional arguments: 'edge_index' and 'edge_features'

In [57]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

model.eval()  # Set the model to evaluation mode
predictions, actuals = [], []
with torch.no_grad():  # No need to track gradients during evaluation
    for batch in test_loader:  # Assume you have a DataLoader `test_loader`
        numerical_data = batch[0]
        targets = batch[1]
        outputs = model(numerical_data).squeeze()
        
        # Store predictions and actual values
        predictions.extend(outputs.tolist())
        actuals.extend(targets.tolist())

# Convert lists to NumPy arrays for evaluation
predictions = np.array(predictions)
actuals = np.array(actuals)

# Calculate evaluation metrics
print(actuals.shape, predictions.shape)
mae = mean_absolute_error(actuals, predictions)
mse = mean_squared_error(actuals, predictions)
rmse = np.sqrt(mse)  # Root Mean Squared Error

print(f'Mean Absolute Error: {mae:.2f}')
print(f'Mean Squared Error: {mse:.2f}')
print(f'Root Mean Squared Error: {rmse:.2f}')


(10459,) (10459,)
Mean Absolute Error: 21.97
Mean Squared Error: 1818.16
Root Mean Squared Error: 42.64


In [ ]:
import dgl

dataset = dgl.data.CiteseerGraphDataset()
graph = dataset[0]

/Users/teframartin/Informatik/Swisscom/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


  NumNodes: 3327
  NumEdges: 9228
  NumFeats: 3703
  NumClasses: 6
  NumTrainingSamples: 120
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [ ]:
graph

Graph(num_nodes=3327, num_edges=9228,
      ndata_schemes={'feat': Scheme(shape=(3703,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'train_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={})

In [ ]:
import numpy as np
import torch

n_users = 1000
n_items = 500
n_follows = 3000
n_clicks = 5000
n_dislikes = 500
n_hetero_features = 10
n_user_classes = 5
n_max_clicks = 10

follow_src = np.random.randint(0, n_users, n_follows)
follow_dst = np.random.randint(0, n_users, n_follows)
click_src = np.random.randint(0, n_users, n_clicks)
click_dst = np.random.randint(0, n_items, n_clicks)
dislike_src = np.random.randint(0, n_users, n_dislikes)
dislike_dst = np.random.randint(0, n_items, n_dislikes)

hetero_graph = dgl.heterograph({
    ('user', 'follow', 'user'): (follow_src, follow_dst),
    ('user', 'followed-by', 'user'): (follow_dst, follow_src),
    ('user', 'click', 'item'): (click_src, click_dst),
    ('item', 'clicked-by', 'user'): (click_dst, click_src),
    ('user', 'dislike', 'item'): (dislike_src, dislike_dst),
    ('item', 'disliked-by', 'user'): (dislike_dst, dislike_src)})

hetero_graph.nodes['user'].data['feature'] = torch.randn(n_users, n_hetero_features)
hetero_graph.nodes['item'].data['feature'] = torch.randn(n_items, n_hetero_features)
hetero_graph.nodes['user'].data['label'] = torch.randint(0, n_user_classes, (n_users,))
hetero_graph.edges['click'].data['label'] = torch.randint(1, n_max_clicks, (n_clicks,)).float()
# randomly generate training masks on user nodes and click edges
hetero_graph.nodes['user'].data['train_mask'] = torch.zeros(n_users, dtype=torch.bool).bernoulli(0.6)
hetero_graph.edges['click'].data['train_mask'] = torch.zeros(n_clicks, dtype=torch.bool).bernoulli(0.6)

In [ ]:
# Contruct a two-layer GNN model
import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F
class SAGE(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats):
        super().__init__()
        self.conv1 = dglnn.SAGEConv(
            in_feats=in_feats, out_feats=hid_feats, aggregator_type='mean')
        self.conv2 = dglnn.SAGEConv(
            in_feats=hid_feats, out_feats=out_feats, aggregator_type='mean')

    def forward(self, graph, inputs):
        # inputs are features of nodes
        h = self.conv1(graph, inputs)
        h = F.relu(h)
        h = self.conv2(graph, h)
        return h

In [ ]:
node_features = graph.ndata['feat']
node_labels = graph.ndata['label']
train_mask = graph.ndata['train_mask']
valid_mask = graph.ndata['val_mask']
test_mask = graph.ndata['test_mask']
n_features = node_features.shape[1]
n_labels = int(node_labels.max().item() + 1)

In [ ]:
graph.ndata["feat"].shape, graph.ndata["label"].shape

(torch.Size([3327, 3703]), torch.Size([3327]))

In [ ]:
def evaluate(model, graph, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(graph, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [ ]:
n_features, n_labels

(3703, 6)

In [ ]:
model = SAGE(in_feats=n_features, hid_feats=100, out_feats=n_labels)
opt = torch.optim.Adam(model.parameters())

for epoch in range(10):
    model.train()
    # forward propagation by using all nodes
    logits = model(graph, node_features)
    # compute loss
    loss = F.cross_entropy(logits[train_mask], node_labels[train_mask])
    # compute validation accuracy
    acc = evaluate(model, graph, node_features, node_labels, valid_mask)
    # backward propagation
    opt.zero_grad()
    loss.backward()
    opt.step()
    print(loss.item())

    # Save model if necessary.  Omitted in this example.

1.7923964262008667
1.7791969776153564
1.7661806344985962
1.7529937028884888
1.7394185066223145
1.7253941297531128
1.7108708620071411
1.695776104927063
1.6801279783248901
1.6639060974121094
